In [1]:
from qondense.tapering import tapering
from qondense.cs_vqe import cs_vqe
from qondense.utils.operator_toolkit import exact_gs_energy, plot_ground_state_amplitudes, number_of_qubits
import qondense.utils.qonversion_tools as qonvert
from qondense.utils.ordering_heuristics import ordering_heuristics
import json
import numpy as np
import openfermion as of
import openfermionpyscf as ofpyscf
from openfermion.circuits import ( uccsd_singlet_get_packed_amplitudes,
                                   uccsd_singlet_generator, uccsd_generator,
                                   uccsd_convert_amplitude_format)
from itertools import combinations

In [2]:
with open('data/molecule_geometries/molecule_data.json') as jfile:
    molecule_geometries = json.load(jfile)
print(molecule_geometries.keys())

dict_keys(['H2_3-21G_SINGLET', 'H6_STO-3G_SINGLET', 'H2_6-31G_SINGLET', 'H2_6-311G_SINGLET', 'H3+_STO-3G_SINGLET', 'H3+_3-21G_SINGLET', 'HeH+_3-21G_SINGLET', 'HeH+_6-311G_SINGLET', 'H2O_STO-3G_SINGLET', 'BeH+_STO-3G_SINGLET', 'LiH_STO-3G_SINGLET', 'CH+_STO-3G_SINGLET', 'HF_STO-3G_SINGLET', 'B+_STO-3G_SINGLET', 'B_STO-3G_DOUBLET', 'N_STO-3G_QUARTET', 'OH-_STO-3G_SINGLET', 'O_STO-3G_TRIPLET', 'CH2_STO-3G_TRIPLET', 'BeH2_STO-3G_SINGLET', 'Be_STO-3G_SINGLET', 'C_STO-3G_TRIPLET', 'NH_STO-3G_SINGLET', 'Ne_STO-3G_SINGLET', 'F_STO-3G_DOUBLET', 'Li_STO-3G_DOUBLET', 'BH_STO-3G_SINGLET', 'NeH+_STO-3G_SINGLET', 'NH2+_STO-3G_SINGLET', 'BH2+_STO-3G_SINGLET', 'HCl_STO-3G_SINGLET', 'H4_STO-3G_SINGLET', 'NH3_STO-3G_SINGLET', 'F2_STO-3G_SINGLET', 'HCN_STO-3G_SINGLET', 'CH4_STO-3G_SINGLET', 'CH3OH_STO-3G_SINGLET', 'C2H6_STO-3G_SINGLET', 'CH3CN_STO-3G_SINGLET', 'CH3CHO_STO-3G_SINGLET', 'CH3CHOHCH3_STO-3G_SINGLET', 'CHONH2_STO-3G_SINGLET', 'CO2_STO-3G_SINGLET', 'O2_STO-3G_SINGLET', 'O3_STO-3G_SINGLET', 'HO

In [3]:
# Set molecule parameters
speciesname = 'H2S_STO-3G_SINGLET'
mol_data = molecule_geometries[speciesname]
if 'name' in mol_data:
    print(mol_data['name'])
    
atoms = mol_data['atoms']
coords = mol_data['coords']
basis = mol_data['basis']
multiplicity = mol_data['multiplicity']
charge = mol_data['charge']
geometry = list(zip(atoms, coords))

delete_input = True
delete_output = True
cisd=1
ccsd=1
fci =1 # wouldn't advise turning this on over 32 qubits!

# Run pyscf.
molecule_data = of.MolecularData(geometry, basis, multiplicity, charge)
calculated_molecule = ofpyscf.run_pyscf(molecule_data,
                     run_scf=1,run_mp2=1,run_cisd=cisd,run_ccsd=ccsd,run_fci=fci)

hydrogen sulfide


In [ ]:
oh = ordering_heuristics(calculated_molecule)

------------------------------------------------
Information concerning the full system:
------------------------------------------------
Number of qubits in full problem: 22
The Hartree-Fock state is |1111111111111111110000>
HF   energy = -394.31155566
MP2  energy = -394.34069092
CISD energy = -394.35381008
CCSD energy = -394.35443220
FCI energy  = -394.35449766
------------------------------------------------
Tapering information:
------------------------------------------------
We are able to taper 4 qubits from the Hamiltonian
The symmetry sector is [-1, -1, 1, 1]
The tapered Hartree-Fock state is |111111111111110000>
------------------------------------------------


In [5]:
oh.ham_dict

{'IIIIIIIIIIIIII': (-46.40178201838026+0j),
 'ZIIIIIIIIIIIII': (12.413911714677482+0j),
 'YZYIIIIIIIIIII': (0.1250642657528705+0j),
 'XZXIIIIIIIIIII': (0.1250642657528705+0j),
 'YZZZZZYIIIIIII': (-0.04270951779188848+0j),
 'XZZZZZXIIIIIII': (-0.04270951779188848+0j),
 'YZZZZZZZZZYIII': (0.07314533074510181+0j),
 'XZZZZZZZZZXIII': (0.07314533074510181+0j),
 'IZIIIIIIIIIIII': (12.413911714677482+0j),
 'IYZYIIIIIIIIII': (0.1250642657528705+0j),
 'IXZXIIIIIIIIII': (0.1250642657528705+0j),
 'IYZZZZZYIIIIII': (-0.04270951779188847+0j),
 'IXZZZZZXIIIIII': (-0.04270951779188847+0j),
 'IYZZZZZZZZZYII': (0.07314533074510178+0j),
 'IXZZZZZZZZZXII': (0.07314533074510178+0j),
 'IIZIIIIIIIIIII': (1.6577822216057907+0j),
 'IIYZZZYIIIIIII': (0.11811415718604289+0j),
 'IIXZZZXIIIIIII': (0.11811415718604289+0j),
 'IIYZZZZZZZYIII': (-0.28534861549870166+0j),
 'IIXZZZZZZZXIII': (-0.28534861549870166+0j),
 'IIIZIIIIIIIIII': (1.6577822216057907+0j),
 'IIIYZZZYIIIIII': (0.11811415718604296+0j),
 'IIIXZZZXIII

In [179]:
from qondense.utils.QubitOp import QubitOp
from qondense.utils.cs_vqe_tools_legacy import contextualQ_ham
#op_list = ['IIYI', 'XYXI', 'XZXI', 'XZZI', 'YXYI', 'ZZZI', 'IIIZ', 'XXXI', 'XXYI', 'XXZI', 'YXXI', 'YYZI', 'YZXI', 'ZYYI']
op_list = ['IIIZ', 'XZXI', 'ZZZI', 'IIYI', 'YXYI', 'XYXI', 'XZZI']
#op_list = ['IX', 'XI', 'XZ', 'ZX']
op = QubitOp(op_list)
op = oh.ham_noncontextual
#op = oh.hamiltonian

adjcmat = op.adjacency_matrix().toarray()
nonuniv = np.ones(op.n_terms, dtype=bool)
univcom = np.where(np.all(adjcmat==0, axis=0))
nonuniv[univcom]=0
remove_univcom = adjcmat[nonuniv,:][:,nonuniv]

#remove duplicate rows/cols
def condense_duplicate_rows(input_array):
    ordering = np.lexsort(input_array.T)
    diff_adjacent = np.diff(input_array[ordering], axis=0)
    uniqueidx = np.append(True, ~np.all(diff_adjacent==0, axis=1))
    condensed = input_array[ordering][uniqueidx]
    return condensed   
reduce_rows = condense_duplicate_rows(remove_univcom)
reduce_cols = condense_duplicate_rows(reduce_rows.T).T

# if noncontextual then identity!
print('Noncontextual?')
print(np.all(np.array(reduce_cols[::-1]==0, dtype=int)==np.eye(*reduce_cols.shape, dtype=int)))

print(not contextualQ_ham(op._dict()))

Noncontextual?
True
True


In [74]:
x = np.arange(0,oh.n_qubits,1)
a = (1-(2*oh.HL_index)/oh.n_qubits)*(x-oh.n_qubits/2)
b = oh.HL_index*np.log(oh.HL_index/oh.n_qubits)*np.exp(-((x-oh.HL_index)**2)/oh.n_qubits)
f = a+b*10/3
g = f/np.min(f)
stab_removal_order = []
for i in np.arange(0,2*oh.n_qubits+1):
    threshold = 1-i/oh.n_qubits
    stab_indices = list(np.where(g>=threshold)[0])
    if stab_indices != []:
        if stab_indices not in stab_removal_order: 
            stab_removal_order.append(stab_indices)
stab_removal_order

[[6],
 [6, 7],
 [5, 6, 7],
 [4, 5, 6, 7],
 [3, 4, 5, 6, 7],
 [2, 3, 4, 5, 6, 7],
 [1, 2, 3, 4, 5, 6, 7],
 [0, 1, 2, 3, 4, 5, 6, 7]]

In [12]:
from itertools import product

supported_Gi={q_pos:[i for i,Gi in enumerate(oh.generators) if Gi[q_pos]=='Z']
             for q_pos in range(oh.n_qubits)}
qubit_order = list(range(oh.HL_index, oh.n_qubits))+list(range(oh.HL_index))[::-1]
#qubit_order = [6,5,7,4,3,2,1,0]
num_sim_q = 14
if num_sim_q <= 5:
    m_type = 'dense'
else:
    m_type = 'sparse'
    
stab_index_pool = [supported_Gi[i] for i in qubit_order[:num_sim_q]]
print(stab_index_pool)

[[1, 3, 7, 17], [2, 4, 10, 12, 18], [1, 3, 7, 19], [2, 4, 10, 12, 18, 20], [1, 3, 7], [2, 4, 10, 12, 18, 21], [1, 3, 7, 22], [2, 4, 10, 12, 18, 23], [2, 4, 10, 12, 16], [1, 3, 7, 15], [2, 4, 10, 12, 14], [1, 3, 7, 13], [2, 4, 10, 12], [1, 3, 7, 11]]


In [ ]:
trial_stab_indices = [list(set(range(oh.n_qubits))-set(stab_indices)) 
                      for stab_indices in list(product(*stab_index_pool)) 
                      if len(set(stab_indices))==num_sim_q]

#print(trial_stab_indices)

In [9]:
for stab_indices in trial_stab_indices:
    print(stab_indices)
    energy, vector = exact_gs_energy(oh.contextual_subspace_hamiltonian(stab_indices), matrix_type=m_type)
    print(energy-oh.fci_energy)

[0, 1, 2, 3, 4, 5, 6]
0.056314661888251294


In [ ]:
print()

In [33]:
print(oh.HL_index, oh.hf_tapered)
qubit_order = list(range(oh.HL_index, oh.n_qubits))+list(range(oh.HL_index))[::-1]
for q_pos in qubit_order:
    print(q_pos)
    supported_Gi = [i for i,Gi in enumerate(oh.generators) if Gi[q_pos]=='Z']
    for i in supported_Gi:
        print(i, oh.generators[i])
        stab_indices=list(set(range(oh.n_qubits))-set([i]))
        num_sim_q = oh.n_qubits-len(stab_indices)
        if num_sim_q <= 5:
            m_type = 'dense'
        else:
            m_type = 'sparse'
        print(exact_gs_energy(oh.contextual_subspace_hamiltonian(stab_indices), matrix_type=m_type)[0])
    print()

6 [1, 1, 1, 1, 1, 1, 0, 0]
6
1 ZIZIZZZI
-98.57101107
3 IIZIZZZI
-98.57101107
7 IIIIIIZI
-98.57101106

7
2 IZIZZZIZ
-98.57101107
4 IIIZZZIZ
-98.57101107

5
1 ZIZIZZZI
-98.57101107
2 IZIZZZIZ
-98.57101107
3 IIZIZZZI
-98.57101107
4 IIIZZZIZ
-98.57101107
6 IIIIIZII
-98.57101107000001

4
1 ZIZIZZZI
-98.57101107
2 IZIZZZIZ
-98.57101107
3 IIZIZZZI
-98.57101107
4 IIIZZZIZ
-98.57101107
5 IIIIZIII
-98.57101107000001

3
0 IIIZIIII
-98.57101107000001
2 IZIZZZIZ
-98.57101107
4 IIIZZZIZ
-98.57101107

2
1 ZIZIZZZI
-98.57101107
3 IIZIZZZI
-98.57101107

1
2 IZIZZZIZ
-98.57101107

0
1 ZIZIZZZI
-98.57101107



In [66]:
methods = ["f"]#["a", "b", "c", "d"]#, "e", "f"]
data = {"methods":methods}
for m in methods:
    print('Method: ',m,'\n------------------------------------------')
    data[m] = oh.heuristic_errors(heuristic=m, print_info=True)

Method:  f 
------------------------------------------
Number of qubits simulated: 1
Ground energy: -98.5710110700
CS-VQE error w.r.t. HF energy:  -0.0000000020
CS-VQE error w.r.t. MP2 energy:  0.0209705856
CS-VQE error w.r.t. CISD energy: 0.0322907073
CS-VQE error w.r.t. CCSD energy: 0.0322907039
CS-VQE error w.r.t. FCI energy:  0.0322907073

Number of qubits simulated: 2
Ground energy: -98.5964896412
CS-VQE error w.r.t. HF energy:  -0.0254785732
CS-VQE error w.r.t. MP2 energy: -0.0045079856
CS-VQE error w.r.t. CISD energy: 0.0068121361
CS-VQE error w.r.t. CCSD energy: 0.0068121327
CS-VQE error w.r.t. FCI energy:  0.0068121361

Number of qubits simulated: 3
Ground energy: -98.5985972242
CS-VQE error w.r.t. HF energy:  -0.0275861562
CS-VQE error w.r.t. MP2 energy: -0.0066155686
CS-VQE error w.r.t. CISD energy: 0.0047045531
CS-VQE error w.r.t. CCSD energy: 0.0047045497
CS-VQE error w.r.t. FCI energy:  0.0047045531

Number of qubits simulated: 4
Ground energy: -98.6024931066
CS-VQE error

In [67]:
data

{'methods': ['f'],
 'f': {'heuristic': 'perfect_removal',
  1: {'energy': -98.57101107000001,
   'error': 0.0322907072588805,
   'stab_indices': [0, 1, 2, 3, 4, 5, 7]},
  2: {'energy': -98.59648964116543,
   'error': 0.006812136093458321,
   'stab_indices': [1, 2, 3, 4, 5, 6]},
  3: {'energy': -98.59859722417421,
   'error': 0.004704553084678764,
   'stab_indices': [1, 2, 4, 5, 6]},
  4: {'energy': -98.60249310656864,
   'error': 0.0008086706902474816,
   'stab_indices': [1, 2, 5, 6]},
  5: {'energy': -98.60290001838425,
   'error': 0.0004017588746449974,
   'stab_indices': [1, 2, 6]},
  6: {'energy': -98.60328182139287,
   'error': 1.9955866022769442e-05,
   'stab_indices': [1, 2]},
  7: {'energy': -98.60328811880703,
   'error': 1.3658451862852417e-05,
   'stab_indices': [2]}}}

In [8]:
data['num_tapered'] = oh.n_taper
data['num_qubits']  = oh.n_qubits
data['stabilizers'] = oh.generators
data['hf_energy']   = oh.hf_energy
data['mp_energy']   = oh.mp_energy
data['cisd_energy'] = oh.cisd_energy
data['ccsd_energy'] = oh.ccsd_energy
data['fci_energy']  = oh.fci_energy

with open('data/cs_vqe_results/'+speciesname+'_heuristic_errors.json', 'w') as outfile:
    json.dump(data, outfile)

In [9]:
df_type = 'B97M'
print(f'List of {df_type} density functionals in PySCF:\n')
for df in dft.libxc.XC_CODES.keys():
    if df.find(df_type)!=-1:
        print(df)

List of B97M density functionals in PySCF:



NameError: name 'dft' is not defined

In [ ]:
from pyscf import gto, dft, scf, mp, fci, cc

atom_geom = list(zip(atoms, coords))
print(atom_geom)

mol_sys = gto.M(atom = atom_geom, charge=0, spin=0, basis = 'sto-3g', symmetry=True, unit='Angstrom')
mol_sys.build()
#mf_hf = dft.RKS(mol_hf)
#mf_hf.xc = 'lda,vwn' # default
#mf_hf.xc = 'b3lyp'
#mf_hf = mf_hf.newton() # second-order algortihm
print('Performing Hartree-Fock calculation:')
mol_HFK = scf.RHF(mol_sys)
hfk_energy = mol_HFK.kernel()
print(hfk_energy, '\n')

print('\nPerforming Møller–Plesset perturbation:')
mol_MP2 = mp.MP2(mol_HFK)
mol_MP2.kernel()

print('\nPerforming coupled-cluster singles doubles calculation:')
mol_CCSD = cc.CCSD(mol_HFK)
mol_CCSD.kernel()

print('\nPerforming density functional calculation:')
mol_DFT=dft.RKS(mol_sys)
mol_DFT.xc = 'MGGA_XC_B97M_V'
mol_DFT.kernel()

#print('\nPerforming full-configuration interaction calculation:')
#mol_FCI = fci.FCI(mol_HFK)
#mol_FCI.kernel()
#print(f'E(FCI) = {mol_FCI.e_tot}')

print('\nElectronic structure calculcations complete')

In [ ]:
mol_HFK.e_tot, mol_MP2.e_tot, mol_CCSD.e_tot, mol_DFT.e_tot#, mol_FCI.e_tot

In [ ]:
oh.hf_energy, oh.mp_energy, oh.fci_energy

In [ ]:
list(zip([1,2,3], [None, None, None]))

In [ ]:
help(mol_hf)

In [ ]:
HF_X, LDA_X = .6, .08
B88_X = 1. - HF_X - LDA_X
LYP_C = .81
VWN_C = 1. - LYP_C
mf_hf.xc = f'{HF_X:} * HF + {LDA_X:} * LDA + {B88_X:} * B88, {LYP_C:} * LYP + {VWN_C:} * VWN'
mf_hf.kernel()
mf_hf.xc = 'hf'
mf_hf.kernel()

In [ ]:
molecule_data.fci_energy